# Parallel multi-point sequential Bayesian optimisation

In this example, NUBO is used to perform multi-point optimisation that allows the candidates to be evaluated from the objective function in parallel. Multi-point optimisation is implemented in NUBO through Monte Carlo acquisition functions. The script below uses the `MCUpperConfidenceBound` acquisition function with 512 samples and resamples the base samples (default). Each batch of 4 is found sequentialy with the `sequential()` function by optimising the acquisition function with the stochastic Adam optimiser. Note that the number of restarts is set to 1 for demonstration only. In practice you should consider a multi-start approach to prevent getting stuck in a local optimum. We could also fix the base samples in `MCUpperConfidenceBound` and use a deterministic optimiser, such as `lbfgsb()` or `slsqp()`. The `Hartmann6D` synthetic test function acts as a surrogate for a black box objective funtion, such as an experiment or a simulation. The optimisation loop is run for 10 iterations returning batches of 4 each (a total of 40 evaluations) and finds a solution close the true optimum of -3.3224.

In [1]:
import torch
from nubo.acquisition import MCExpectedImprovement, MCUpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import sequential
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 10

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = MCExpectedImprovement(gp=gp, y_best=torch.max(y_train), samples=256)
    acq = MCUpperConfidenceBound(gp=gp, beta=1.96**2, samples=512)

    # optimise acquisition function
    x_new, _ = sequential(func=acq, method="Adam", batch_size=4, bounds=bounds, lr=0.1, steps=200, num_starts=1)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if torch.max(y_new) > torch.max(y_train[:-y_new.size(0)]):
        best_eval = torch.argmax(y_train)
        print(f"New best at evaluation {best_eval+1}: \t Inputs: {x_train[best_eval, :].numpy().reshape(dims).round(4)}, \t Outputs: {-y_train[best_eval].numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {-float(y_train[best_iter]):.4f}")


New best at evaluation 31: 	 Inputs: [0.2376 0.0057 0.6379 0.1616 0.3322 0.6163], 	 Outputs: -2.4684
New best at evaluation 37: 	 Inputs: [1.686e-01 5.000e-04 4.184e-01 2.062e-01 3.457e-01 5.791e-01], 	 Outputs: -2.7252
New best at evaluation 46: 	 Inputs: [0.2303 0.     0.4441 0.3053 0.2899 0.6732], 	 Outputs: -3.0165
New best at evaluation 47: 	 Inputs: [0.2088 0.0903 0.4907 0.3055 0.3147 0.6268], 	 Outputs: -3.2279
New best at evaluation 51: 	 Inputs: [0.1872 0.1524 0.475  0.2819 0.3088 0.6426], 	 Outputs: -3.3123
Evaluation: 51 	 Solution: -3.3123
